<a href="https://colab.research.google.com/github/arafat04/bn-hi-MT-improvement-using-llm/blob/main/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import sacrebleu.utils
from sacrebleu.metrics.bleu import BLEU
from tqdm import tqdm
import transformers
import torch

In [ ]:
template = """{source_lang}: {source_text}
{target_lang}: {target_text}"""

def apply_prompt(training=False, eos_token=None, **kwargs):
    # note: we strip because of potential trailing whitespace
    # we also provide a default value for target_text so that it can be omitted
    return template.format(**{"target_text": "", **kwargs}).strip() + ("" if not training or eos_token is None else eos_token)

def apply_prompt_n_shot(examples, n: int, eos_token: str, **kwargs):
    return (eos_token + "\n\n").join(
        [apply_prompt(**{"target_text": "", **example}) for example in examples[:n]] + [apply_prompt(**kwargs)]
    )

EXAMPLE_SENTENCES = [
    {
        "source_lang": "Bengali",
        "target_lang": "Hindi",
        "source_text": "খবরটা শুনে খুব খারাপ লাগলো।",
        "target_text": "खबर सुनकर बहुत दुख हुआ.",
    },
    {
        "source_lang": "Bengali",
        "target_lang": "Hindi",
        "source_text": "এটার দাম কত?",
        "target_text": "इसकी कीमत कितनी होती है?",
    },
    {
        "source_lang": "Bengali",
        "target_lang": "Hindi",
        "source_text": "ঢাকা বাংলাদেশের রাজধানী।",
        "target_text": "ढाका बांग्लादेश की राजधानी है.",
    },
    {
        "source_lang": "Bengali",
        "target_lang": "Hindi",
        "source_text": "রাস্তার দিকে মনোযোগ দিন।",
        "target_text": "सड़क पर ध्यान दें.",
    },
    {
        "source_lang": "Bengali",
        "target_lang": "Hindi",
        "source_text": "আমার মাথা ব্যথা করছে",
        "target_text": "मुझे सिर दर्द है.",
    }
]

In [ ]:
print(sacrebleu.utils.get_source_file("wmt21", "bn-hi"))
with open(sacrebleu.utils.get_source_file("wmt21", "bn-hi"), "r", encoding="utf-8") as fd:
    sources = list(map(str.strip, fd.readlines()))
with open(sacrebleu.utils.get_reference_files("wmt21", "bn-hi")[0], "r", encoding="utf-8") as fd:
    references = list(map(str.strip, fd.readlines()))
source_lang = "Bengali"
target_lang = "Hindi"

/storage/praha2-natur/home/rahmang/.sacrebleu/wmt21/wmt21.bn-hi.src


In [ ]:
from transformers import StoppingCriteria
class EosListStoppingCriteria(StoppingCriteria):
    # Adopted from: https://github.com/huggingface/transformers/issues/26959
    def __init__(self, eos_sequence = [13]):  # Stop on newline
        self.eos_sequence = eos_sequence

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        last_ids = input_ids[:,-len(self.eos_sequence):].tolist()
        return self.eos_sequence in last_ids

def translate(model, tokenizer, source_lang, target_lang, source_texts: list[str], n_shot: int = 0):
    prompts = [apply_prompt_n_shot(EXAMPLE_SENTENCES, n_shot, eos_token=tokenizer.eos_token, source_lang=source_lang, target_lang=target_lang, source_text=source_text) for source_text in source_texts]

    translations = []
    for prompt in prompts:
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True, stopping_criteria=[EosListStoppingCriteria()])
        decoded = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()
        translations.append(decoded)

    return translations

def evaluate(model, tokenizer, n_shot: int = 0):
    translations = translate(model, tokenizer, source_lang, target_lang, sources, n_shot=n_shot)
    return BLEU().corpus_score(translations, [references])

In [ ]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "outputs/mistral-ft-qlora",
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)


/storage/praha1/home/rahmang/envs/unsloth/lib/python3.10/site-packages/unsloth/__init__.py:22: UserWarning: Unsloth: 'CUDA_VISIBLE_DEVICES' is currently GPU-044e48b9-e794-cbfc-316e-1472ca268b72 but we require 'CUDA_VISIBLE_DEVICES=0'
We shall set it ourselves.
  warnings.warn(
/storage/praha1/home/rahmang/envs/unsloth/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: NVIDIA RTX A4000. Max memory: 15.724 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/storage/praha1/home/rahmang/envs/unsloth/lib/python3.10/site-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Unsloth 2024.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
sources[:5]

['ইকুয়েডরীয় গ্যালোপেগোস দ্বীপপুঞ্জের এক নতুন প্রজাতির পাখির গঠন নিয়ে জার্নাল সায়েন্স রিপোর্টে বৃহস্পতিবার একটি গবেষণাপত্র প্রকাশিত হয়েছে।',
 'মার্কিন যুক্তরাষ্ট্রের প্রিন্সটন বিশ্ববিদ্যালয় এবং সুইডেনের ইউপসালা বিশ্ববিদ্যালয়ের গবেষকরা জানিয়েছেন যে নতুন প্রজাতিটি শুধুমাত্র দুটি প্রজন্মের মধ্যে বিবর্তিত হয়েছে, যদিও ডারউইন ফিঞ্চ, জেসোপিজা ফোর্টস এবং ইমিগ্র্যান্ট ক্যাকটাসের ফিঞ্চ, জিওপিজা কনিরোস্ট্রিস মধ্যে প্রজননের কারণে এই প্রক্রিয়াটি আরও বেশি সময় নেয় বলে মনে করা হয়।',
 '"এই বিতর্কটি হারিকেন ক্যাটরিনার সময় ত্রাণ ও পুনর্নির্মাণের উপর ব্যয়কে কেন্দ্র করে বেশি ছড়িয়েছিল; যেটাকে কিছু অর্থ সংশ্লিষ্ট রক্ষণশীলরা হাস্যকরভাবে ""বুশের নিউ অরলিন্স চুক্তি"" বলে অবহিত করেছিল।"',
 'পুনর্নির্মাণের চেষ্টা সম্পর্কে উদার সমালোচনা ওয়াশিংটন অভ্যন্তরীণদের পুনর্নির্মাণ চুক্তিগুলি অনুধাবনের দিকে লক্ষ্যস্থির করেছে।',
 'খেলাটি দুর্দান্ত আবহাওয়ার সাথে সকাল ১০:০০ টায় শুরু হয়েছিল এবং মধ্য সকালের দ্রুত পরিষ্কার হয়ে যাওয়া হালকা বৃষ্টিপাত ছাড়া, এটি সপ্তম রাগবির জন্য একটি উপযুক্ত দিন ছিল।']

In [ ]:
translate(model, tokenizer, source_lang, target_lang, sources[:5])

['इकवेडरी ग्यालोपेगोस द्वीपपुंज के एक नया प्रजाति के लिए एक नया पक्ष का प्रतिक्रिया करने के लिए जार्नल सायन्स रिपोर्ट को बृहस्पतिवार एक गवेषणापत्र प्रकाशित हुआ है.',
 'मार्किन यूक्तराष्ट्र के प्रिन्सटन विश्वविद्यालय और स्वीडन के यूपसाला विश्वविद्यालय के अनुसार नयी प्रजाति केवल दो प्राणी के बीच बदला हुआ है, जो दुई प्राणियों के बीच बदला हुआ है, जो डारूइन फिंच, जेसोपिजा फोर्टस और इमिग्रेंट कैकटास के फिंच, जीपिजा कोनिरो',
 '"Ee vitarakti haarikena kaatarin samay traan aur punirnaan ke upar vyayakay ke kendra karne ke liye bhi bhariyayi thi; ye keechee arth sangshlisht raakshanashilaraa hasyakar bhav se "bushar new orlins chukti" bolte hain."',
 'पुनर्निर्माण के संबंध में वाशिंगटन अभ्यन्तरीणों के पुनर्निर्माण चुक्तियों को पुनर्निर्माण के लिए लक्ष्य स्थिर करने के लिए प्रयोग किया गया है।',
 'खेलाटी दुर्दान्त आभावार साथे सकाल 10:00 तक शुरू हुआ और मध्य सकाल के द्रुत परिष्कार होने के कारण, यह सप्तमी रागवीर के लिए एक उपयुक्त दिन था.']

In [ ]:
evaluate(model, tokenizer, n_shot=0)  # note: took ~79min
#BLEU = 5.88 32.7/9.8/3.3/1.1 (BP = 1.000 ratio = 1.038 hyp_len = 14355 ref_len = 13829)

BLEU = 5.88 32.7/9.8/3.3/1.1 (BP = 1.000 ratio = 1.038 hyp_len = 14355 ref_len = 13829)

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-bnb-4bit",
    max_seq_length=4096,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)
evaluate(model, tokenizer, n_shot=5)

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.581 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.24. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

KeyboardInterrupt: 